In [7]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [8]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [10]:
#MODEL
list_i = []
list_return_price = []
list_rmse = []
list_mae =[]
list_mse = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    model = Lasso(alpha=1.0)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {mse:.2f}',f' RMSE: {mse:.2f}', f'MAE: {mae:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -1.11  R2_train: 0.43  R2_test: 0.51  RMSE: 2.32  RMSE: 2.32 MAE: 1.09
Stock: BCM  Predict: -0.46  R2_train: 0.26  R2_test: 0.35  RMSE: 0.77  RMSE: 0.77 MAE: 0.60
Stock: VHM  Predict: 0.35  R2_train: 0.21  R2_test: 0.19  RMSE: 4.00  RMSE: 4.00 MAE: 1.39
Stock: VIC  Predict: 0.24  R2_train: 0.18  R2_test: 0.14  RMSE: 3.60  RMSE: 3.60 MAE: 1.03
Stock: VRE  Predict: -0.67  R2_train: 0.33  R2_test: 0.43  RMSE: 1.82  RMSE: 1.82 MAE: 0.90
Stock: BVH  Predict: 0.68  R2_train: 0.34  R2_test: 0.56  RMSE: 0.61  RMSE: 0.61 MAE: 0.52
Stock: POW  Predict: 0.89  R2_train: 0.38  R2_test: 0.52  RMSE: 1.32  RMSE: 1.32 MAE: 0.81
Stock: GAS  Predict: 0.68  R2_train: 0.34  R2_test: 0.61  RMSE: 0.45  RMSE: 0.45 MAE: 0.49
Stock: ACB  Predict: 0.47  R2_train: 0.23  R2_test: 0.34  RMSE: 1.09  RMSE: 1.09 MAE: 0.67
Stock: BID  Predict: 0.70  R2_train: 0.32  R2_test: 0.40  RMSE: 1.65  RMSE: 1.65 MAE: 0.85
Stock: CTG  Predict: 0.82  R2_train: 0.35  R2_test: 0.46  RMSE: 1.34  RMSE: 1.34 MAE: 0

In [11]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test, 'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,})
KQ.to_excel("Kết quả 20% test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE
0,SSI,[-1.1063394672454003],0.434865,0.506568,2.318181,1.522557,1.089181
1,BCM,[-0.46468910418299525],0.258901,0.346094,0.767437,0.876035,0.595351
2,VHM,[0.3457842860710538],0.205847,0.193405,3.997530,1.999383,1.392603
3,VIC,[0.23815860828943944],0.182543,0.144241,3.595919,1.896291,1.032087
4,VRE,[-0.6736729538868229],0.332520,0.432370,1.823763,1.350468,0.896925
...,...,...,...,...,...,...,...
389,TDP,[0.49644242867806865],0.227608,0.286028,4.242879,2.059825,1.447155
390,NO1,[-0.099004567788017],0.127151,0.039696,11.217182,3.349206,2.508842
391,L10,[1.4744221702082092],0.506397,0.467755,19.284542,4.391417,3.447319
392,REE,[0.734583591731544],0.294423,0.467943,0.967259,0.983493,0.708654
